# 主题向量

通过隐形语义分析（latent senmantic analysis LSA）可以不仅仅把词的意义表示为向量，还可以用向量来表示通篇文档的意义。

本章将学习这些语义或主题向量，通过TF-IDF向量的加权得分来计算所谓的主题得分，而将这些得分构成了主题向量的各个维度。

将使用归一化词频直接的关联来将词归并到同意主题，每个归并结果定义了新主题向量的一个维度。

In [22]:
import numpy as np 
import random 
topic={}
random.seed(1)
tfidf= dict(list(zip('cat dog apple lion NYC love'.split(),np.random.rand(6))))

print("得到虚拟的每个词的tf-idf值")
tfidf
topic['petness']=(0.3*tfidf['cat']+0.3*tfidf['dog']+0*tfidf['apple']+0*tfidf['lion']-0.2*tfidf['NYC']+0.2*tfidf['love'])
topic['animalness']=(0.1*tfidf['cat']+0.1*tfidf['dog']-0.1*tfidf['apple']+0.5*tfidf['lion']+0.1*tfidf['NYC']-0.1*tfidf['love'])
topic['cityness']=(0*tfidf['cat']-0.1*tfidf['dog']+0.2*tfidf['apple']-0.1*tfidf['lion']-0.5*tfidf['NYC']+0.1*tfidf['love'])
print(topic)
# 构建相应矩阵 
topic_m=np.zeros(shape=(3,6))
print(topic_m)
word_tf=np.zeros(shape=(6,1))
print(word_tf)

得到虚拟的每个词的tf-idf值
{'petness': 0.3801465633219733, 'animalness': 0.4797383032984889, 'cityness': -0.2865590942806303}
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]


#LDA 
LDA 分类器是一种有监督算法，因此需要对文本进行标注，但是其需要训练的样本数相对较少。

LDA是一维模型，所以其不需要SVD，可以只计算二类问题（如垃圾和非垃圾）问题中的每一类的所有TF-IDF向量的质心（平均值）。推导就变成了这两个质心之间的直线，TF-IDF向量与这条直线越近（TF-IDF向量与这两条直线的点积）就表示它与其中一个类接近。

^C


In [14]:
from cgitb import handler
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model_selection import cross_val_score
import numpy as np
print("处理训练数据：...\n")
train_txt = pd.read_table('sms/train.txt',sep='\t',header=None)  
train_txt.columns = ['label', 'text']
label_map = {'ham': 0, 'spam': 1 }#1为垃圾短信
train_txt['label'] = train_txt['label'].map(label_map)

#train_txt = pd.get_dummies(train_txt, columns=['label'])# 将标签onehot编码

def pre_clean_text(origin_text):
    # 去掉标点符号和非法字符
    text = re.sub("[^a-zA-Z]", " ", origin_text)
    # 将字符全部转化为小写，并通过空格符进行分词处理
    words = text.lower().split()

    # 将剩下的词还原成str类型
    cleaned_text = " ".join(words)
    
    return cleaned_text

if __name__=='__main__':

    #清理数据
    train_txt['text'] = train_txt['text'].apply(lambda x: pre_clean_text(x))

    #删去空值.测试时若无效词删去后为空则直接为垃圾信息(实际测试中没有)
    #print(train_txt.shape)
    train_txt = train_txt.loc[train_txt['text'] != '',:]
    # 查看数据
     
    #print(train_txt.shape)
    #实现tf-id数据向量化
    
    tfidf = TfidfVectorizer (
    analyzer="word",
    tokenizer=None,
    preprocessor=None,
    stop_words=None,
    max_features=200)
    word_vict=tfidf.fit_transform(train_txt['text']).toarray()
    print(word_vict.shape)
    print(word_vict[20,:])
    
    mask=np.array(train_txt['label'].astype(bool))
    print("得到{}矩阵".format(mask.shape))
    spam_centroid=word_vict[mask].mean(axis=0).round(2)#axis=0 计算列平均值
    print("垃圾短信平均向量：",spam_centroid.shape)
    ham_centroid=word_vict[mask].mean(axis=0).round(2)
 
    print("短信平均向量：",ham_centroid.shape)
    sh=spam_centroid-ham_centroid
    print(sh.shape)
    spamsocre=word_vict@sh 
   
    spamscore2=word_vict@ham_centroid
     
    from sklearn.preprocessing import MinMaxScaler 
    spam1=MinMaxScaler().fit_transform(spamsocre.reshape(-1,1))#reshape(-1,1)转换成1列：
 
    spam2=MinMaxScaler().fit_transform(spamscore2.reshape(-1,1))
  
    train_txt['lda_score']=spam1
    train_txt['lda_pred']=(train_txt['lda_score']>0.2).astype(int)
    train_txt

处理训练数据：...

(5161, 200)
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.6447955  0.         0.
 0.47170963 0.         0.         0.         0.35643069 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.       

# 隐性语义分析(LSA,Latent Semantic Analysis)
> 参考
> * https://zhuanlan.zhihu.com/p/46376672
> * https://zhuanlan.zhihu.com/p/144367432

LSA的底层是SVD技术，利用SVD将TF_IDF矩阵分解3个矩阵，而后根据其方差贡献率（信息载荷）进行降维，当在NLL中使用SVD时，将其称为隐性语义分析（LSA），

LSA揭示了被隐含并等待被发现的词的语义或意义。

LSA是一种属性技术，用于寻找对任意一组NLP向量进行最佳线性变换（旋转和拉伸）的方法，这些NLP向量包括TF-IDF向量或词袋向量。对许多应用而言，最好的变换方法是将

坐标轴（维度）对齐到新向量中，使得其在词频上具有最大方差。然后可以在新向量空间中去掉哪些对不同文档向量贡献不大的维度。

<font color='red'>LSA 中单词-文本-svd关系</font>
<img src="https://pic2.zhimg.com/v2-b3eb29a45d1fc11f57b858fd5af7571d_r.jpg"/>
> LSA 步骤
1. 构建TF-IDF或其他文档-词矩阵向量,行为文档(doc)，列为词(term)
<img src="https://pic4.zhimg.com/80/v2-288292d4fd98b748c4b5e37786c06bd3_720w.jpg"/>
2. 对矩阵向量进行SVD分解
3. 根据主题数目，或方差贡献率累计比，选择降维数目
4. 对原有矩阵进行降维


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
 

In [ ]:
!pip install nltk

In [14]:
!pip install jieba

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     -------------------------------------- 19.2/19.2 MB 361.1 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314459 sha256=1120f87868820b5f1c4d22cd9bf9a07c0f8622ce0e42b2a1f7ffbb599c2251b8
  Stored in directory: c:\users\tomis\appdata\local\pip\cache\wheels\f3\30\86\64b88bf0241f0132806c61b1e2686b44f1327bfc5642f9d77d
Successfully built jieba


In [22]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd 
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer as tf
import numpy as np 
from sklearn.decomposition import TruncatedSVD
import jieba 

def loadData():
    '''实现载入sklearn中的“20 Newsgroup”数据
    '''
    dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
    return pd.DataFrame(dataset.data)

 
def clearData(df:pd.DataFrame):
    """开始之前，我们先尝试着清理文本数据。主要思想就是清除其中的标点、数字和特殊字符。之后，我们需要删除较短的单词，因为通常它们不会包含什么有用的信息。最后，我们将文本变为不区分大小写。

    Args:
        df (pd.DataFrame): _原始文本_
    """
# removing everything except alphabets`
    
    df['clean_doc'] = df['document'].str.replace("[^a-zA-Z#]", " ")
# removing short words
    df['clean_doc'] = df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# make all text lowercase
    df['clean_doc'] = df['clean_doc'].apply(lambda x: x.lower())
    return df

def cutChineseWord(data,stopwordfile):
  
 
    stopwords = [line.strip() for line in open(stopwordfile, 'r',encoding='utf-8').readlines()]
     ## 首先实现切词
    for i in range(len(data)):
        doc=data.iloc[i-1,0]
        seq_list=jieba.cut(doc,use_paddle=True)
        token=[]
    
        for seq in seq_list:
            if seq not in stopwords:
                token.append(seq)
        
        data.at[i,'words']=list(token)  # 每次在words列第i个位置插入对象list
    return data
 

def stopWords(df:pd.DataFrame):
    """之后我们要删除没有特别意义的停止词，例如“it”、“they”、“am”、“been”、“about”、“because”、“while”等等。为了实现这一目的，我们要对文本进行标记化，也就是将一串文本分割成独立的标记或单词。删除停止词之后，再把这些标记组合在一起。

    Args:
        df (pd.DataFrame): _description_
    """
    stop_words = stopwords.words('english')
    # tokenization分词
    tokenized_doc = df['clean_doc'].apply(lambda x: x.split())
    # remove stop-words
    tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# de-tokenization
    detokenized_doc = []
    for i in range(len(df)):
        t = ' '.join(tokenized_doc[i])
        detokenized_doc.append(t)

    df['clean_doc'] = detokenized_doc
    return df  

def vec_words(document:pd.DataFrame,max_feature:int=200)->np.array:
    """基于sklearn实现文档向量化，其中元素为tf-idf，注意TfidfVectorizer参数

    Args:
        document (pd.DataFrame): _description_

    Returns:
        _type_: _description_
    """
    tfidf =tf(analyzer="word",tokenizer=None,max_features=max_feature)
    tfidf.fit(document)
    lexcion=tfidf.vocabulary_ # 返回向量化对应的词典（字典形式）
    lexcion_len=tfidf.vocabulary_.__len__()  # 返回向量化对应的词典长度
    vec=tfidf.fit_transform(document).toarray()
    df=pd.DataFrame(vec,columns=list(lexcion.keys()))
    return vec,lexcion,lexcion_len,df,tfidf

def svd(vec:np.array):
    """基于np中线性代数函数实现svd，注意np.linalg.svd（）返回的特征值矩阵是一个向量而非矩阵，这是由于方法自动缩减了特征值
    矩阵中0值，k的大小为A(m*n)中最小的值，为此本方法将特征值矩阵还原为m*n大小的矩阵其中k个特征值构成其对角值。

    Args:
        vec (np.array): _description_

    Returns:
        _type_: _description_
    """
     
     
    u,sigma,vt=np.linalg.svd(vec)
    
    Sigma=np.zeros(np.shape(vec))
    len_ar=len(sigma)
    Sigma[:len_ar,:len_ar]=np.diag(sigma)
    return u,Sigma,vt,sigma


def RightdemsionRe(A:np.array,u:np.array,Sigma:np.array,k:int)->np.array:
    """_实现基于SVD的文本矩阵降维,由于是进行主题降维度、所以采用左奇异矩阵将维，实现词的降维_
        SVD 中左（行）奇异矩阵降维 为 B=Ut（k）*A  其中Ut为U左奇异矩阵转置，k为转置后保留的行数

    Args:
        A(np.array):_原文档-词矩阵_
        u (np.array): _左奇异矩阵_
        Sigma (np.array): _奇异值矩阵_
        vt (np.array): _右奇异矩阵_
        n (int): _降维数目（话题数）_

    Returns:
        np.array: _description_
    """
    B=u.T[:k,:]@A
    #B=u[:,:n]@Sigma[:n,:n]@vt[:n,:]
    
    return B 

In [20]:
if __name__=="__main__":
    print("转为Data数据观察")
    documents=loadData()
    print(documents.head(3))
    print(documents.describe())
    #clear data
    documents.columns=['document']
    documents= clearData(documents)
    print(documents.head(3))
    # tf-idf 向量化 
    vec,lexcion,lexcion_len,df,tfidf=vec_words(documents['clean_doc'])
    print(type(lexcion))
    print("得到向量化矩阵形状(行：文本，列：词）：{},词典个数：{}".format(vec.shape,lexcion_len))
    vect=vec.T# 对词向量转置
    print("得到向量化矩阵形状(行：词，列：文本）：{},词典个数：{}".format(vect.shape,lexcion_len))
    print("词典 \n",lexcion)
    #print("向量化后文本(行：文本，列：词\n ",vec[:8,:])
    print("向量化后的文本矩阵(行：文本，列：词）：",df)
    print("在进行svd之前，由于我们的最终目标是进行主题降维，使用np.linga.svd构建u，sigma,vt时候 \n u对应的行向量就应该是其词（最初主题），故应该先对文本向量进行转置，输入（行：词，列：文本）矩阵，\n 在基础上进行左奇异矩阵降维")

    print("由于输入的向量化文本为行：text，列：word，但np.linalg.svd对应的原始矩阵为，行：word，列：text故需要先转置")
    u,Sigma,vt,sigma=svd(vect)
    print('sigma:',sigma.shape)
    print("svg后的左奇异矩阵大小:{},奇异值矩阵大小:{},右奇异矩阵大小：{},奇异矩阵中对角线奇异值数量:{}".format(u.shape,Sigma.shape,vt.shape,sigma.shape))
    B=RightdemsionRe(vect,u,Sigma,20)
    print("降维后数据大小：",B.shape)
    print(B[:,15])
    
    ### 通过sklearn中TruncatedSVD函数实现svd，但注意，其输入向量矩阵为（行：文本，列：词）
    svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)

    sk_vec=svd_model.fit(vec)
    print("使用sklearn中TruncatedSVD进行降维，注意矩阵形态")
    print(len(svd_model.components_))
    
    '''
    svd_model中的元素就是我们的主题，
    我们可以用svdmodel.components查看。最后，在20个主题中输入几个重要单词，看模型会做出什么反应。
    '''
    terms=tfidf.get_feature_names()
    print(terms)
    print("#############################")
    print(lexcion)
    
    # 查看主题中信息
    for i, comp in enumerate(svd_model.components_):
        terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
    print("Topic "+str(i)+": ")
    for t in sorted_terms:
        print(t[0])
        print(" ")
        

转为Data数据观察
                                                   0
0  Well i'm not sure about the story nad it did s...
1  \n\n\n\n\n\n\nYeah, do you expect people to re...
2  Although I realize that principle is not one o...
            0
count   11314
unique  10994
top          
freq      218


C:\Users\tomis\AppData\Local\Temp\ipykernel_24216\3207423743.py:24: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clean_doc'] = df['document'].str.replace("[^a-zA-Z#]", " ")


KeyboardInterrupt: 

In [6]:
!pip install openpyxl

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/7b/60/9afac4fd6feee0ac09339de4101ee452ea643d26e9ce44c7708a0023f503/openpyxl-3.0.10-py2.py3-none-any.whl (242 kB)


In [23]:
from multiprocessing.resource_sharer import stop
import pandas as pd
if __name__=='__main__':
    df=pd.read_excel("data/TOP25.xlsx")
    
    data=df['workorder_content'].astype(str)#要保证切词前数据全部为str
    stopwordfile='data/baidu_stopwords.txt'
    print(data.head(20))
    # 数据EDA
    print("缺失行数：",data.isnull().sum())
    
    data_clear=cutChineseWord(data,stopwordfile=stopwordfile)


0     斗篷山谷江村八组河边有电线漏电马被电死的问题，公安部门已前往现场，请你部门立即与群众联系核实...
1     本人罗国喜在306监狱跟黔南州建开塔吊到现在得不到工资，请你部门立即与群众联系核实，并反馈完...
2     群众称：小围寨河阳乡蒲林村6组占用其土地未得到赔偿款，要强挖土地，请你部门立即与报警人联系前...
3     群众反映称：在都匀市平桥北路中房停水，请你部门立即与群众联系核实，并反馈完整的处理结果。（供...
4      群众称： 在东山隧道路口的住户供水不正常问题，请你部门立即与群众联系核实，并反馈完整的处理结果。
5      报警人称：在小围寨剑中社区水管被烧断漏水的问题，请你部门立即与群众联系核实，并反馈完整的处理结果
6     群众称：在老师院附中对面工地噪音扰民，请你部门立即与群众联系核实，并反馈完整的处理结果。_x...
7              群众称在三中旁边工地噪音扰民，请你部门立即与群众联系核实，并反馈完整的处理结果。
8     在马鞍山安置房A区楼上亮丽工程灯一直不关影响居民休息的问题，请你部门立即与群众联系核实，并反...
9     群众称：在黔南电视台门口井盖被大货车压碎了存在安全隐患。请你部门立即与群众联系核实，并反馈完...
10    黄女士称：开发区龙彩花园小区水管爆裂几天了，一直往外流水，很浪费，之前工作人员去看过，但是一...
11    报警人称：从夜市街到啤酒厂手机遗失在出租车上，已报警调监控。请你部门立即与群众联系核实，并反...
12        群众称：在裤裆街商家占道经营导致交通堵塞的问题，请你部门立即前往处置并反馈完整的处理结果。
13      报警人称：老火车站环球中心工地施工噪音扰民，请你部门立即与群众联系核实，并反馈完整的处理结果。
14    报警人称打车从大十字到庆云宫，手机遗失在出租车上，记不得车牌，已转接都匀公安110.请你部门...
15       群众称：在东来尚城桥梁厂有人烧垃圾味道重，请你部门立即与群众联系核实，并反馈完整的处理结果。
16           民警称：在都匀一号停车收费的问题，请你部门立即与民警联系核实，并反馈完整的处理结果。
17    群众反映称：在都匀市南洲国际马鞍山小区工地噪音扰民，中午也施工，晚上也施工严重影响孩

IndexingError: Too many indexers

In [ ]:
from collections.abc import Mapping
from nlpia.book.examples.ch04_catdog_lsa_3x6x16 import word_topic_vectors
print(word_topic_vectors.shape)